In [3]:
# merge predictions 
import pandas as pd
import numpy as np

def get_id_modCMA_conf(elitist, mirrored, base_sampler, weights_option, local_restart, step_size_adaptation):
    df_grid = pd.read_csv(f'./data/raw_data/modCMA_conf_grid.csv', index_col=0)
    df_grid = df_grid.replace({np.nan: 'N'})
    if mirrored == 'None':
        mirrored = 'N'
    if local_restart  == 'None':
        local_restart = 'N'
    conf = df_grid[(df_grid['elitist']==elitist)&(df_grid['mirrored']==mirrored)&
    (df_grid['base_sampler']==base_sampler)&(df_grid['weights_option']==weights_option)&
    (df_grid['local_restart']==local_restart)&(df_grid['step_size_adaptation']==step_size_adaptation)].index[0]
    if conf<0 or conf>323:
        print("ERROR")
    else:
        return(conf)

def get_id_modDE_conf(mutation_base,mutation_reference,mutation_n_comps,use_archive,crossover,adaptation_method,lpsr):
    df_grid = pd.read_csv(f'./data/raw_data/modDE_conf_grid.csv', index_col=0)
    df_grid = df_grid.replace({np.nan: 'N'})
    if mutation_reference == 'None':
        mutation_reference = 'N'
    if adaptation_method  == 'None':
        adaptation_method = 'N'
    # print(mutation_base,mutation_reference,mutation_n_comps,use_archive,crossover,adaptation_method,lpsr)
    conf = df_grid[(df_grid['mutation_base']==mutation_base)&(df_grid['mutation_reference']==mutation_reference)&
    (df_grid['mutation_n_comps']==mutation_n_comps)&(df_grid['use_archive']==use_archive)&
    (df_grid['crossover']==crossover)&(df_grid['adaptation_method']==adaptation_method)
    &(df_grid['lpsr']==lpsr)].index[0]
    if conf<0 or conf>576:
        print("ERROR")
    else:
        return(conf)

for modAlgo in ['modDE', 'modCMA']:
    for mode in [ 'p2v', 's2v']:
        for dim in [5, 30]:
            for budget in  [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]:
                df_merged = None
                for module in ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation'] if modAlgo == 'modCMA' else ['mutation_base','mutation_reference','mutation_n_comps','use_archive','crossover','adaptation_method','lpsr']:
                    df = pd.read_csv(f"./results/{modAlgo}/classification_predictions/{mode}_RF_dim_{dim}_budget_{budget}_module_{module}.csv", index_col=0)
                    df.columns = ['true-'+module, 'pred-'+module]
                    
                    if df_merged is None:
                        df_merged = df
                    else:
                        df_merged = df_merged.join(df)
                
                predicted_conf = []
                for index, row in df_merged.iterrows():   
                    if modAlgo == 'modCMA':
                        conf_id = get_id_modCMA_conf(row['pred-elitist'], row['pred-mirrored'],  row['pred-base_sampler'], row['pred-weights_option'], row['pred-local_restart'], row['pred-step_size_adaptation']) 
                    else:
                        conf_id = get_id_modDE_conf(row['pred-mutation_base'], row['pred-mutation_reference'],
                          row['pred-mutation_n_comps'], row['pred-use_archive'],
                           row['pred-crossover'], row['pred-adaptation_method'], row['pred-lpsr']) 
                    
                    predicted_conf.append(conf_id)
                df_merged['predicted_conf'] = predicted_conf 
                df_merged.to_csv(f'./results/{modAlgo}/classification_predictions/merged_{mode}_RF_dim_{dim}_budget_{budget}.csv')


In [6]:
import math

from numpy.linalg import norm
for modAlgo in [ 'modCMA', 'modDE']:
    for mode in ['s2v', 'p2v']:
        for dim in [5, 30]:
            for budget in [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]:
                df = pd.read_csv(f'./results/{modAlgo}/classification_predictions/merged_{mode}_RF_dim_{dim}_budget_{budget}.csv', index_col=0)
                df = df['predicted_conf']
                # df['true_conf'] = df.index
                df.index.name = 'true_conf'
                print(df.head)
                df.to_csv(f"./statistical_tests/true_predicted/{modAlgo}_{mode}_{dim}_{budget}.csv")
                # performance_sim = []
                # cosine_sim = []
                # pearson_corr = []
                # for index, row in df.iterrows():
                #     true_conf = index
                #     predicted_conf = row['predicted_conf']
                #     true_conf_performance = pd.read_csv(f"./data/processed_data_{modAlgo}/performance_data/budget_{budget}_conf_{true_conf}_{dim}D.csv", index_col=0)
                #     predicted_conf_performance = pd.read_csv(f"./data/processed_data_{modAlgo}/performance_data/budget_{budget}_conf_{predicted_conf}_{dim}D.csv", index_col=0)
                #     predicted_conf_performance.columns=['target_p']
                #     df_n = true_conf_performance
                #     df_n = df_n.join(predicted_conf_performance)
                #     true_conf_performance = df_n['target']
                #     predicted_conf_performance = df_n['target_p']
                #     sum = 0
                #     for i in range(0, len(true_conf_performance)):
                #         t = true_conf_performance[i]
                #         p = predicted_conf_performance[i]
                #         sum = sum + pow(t-p, 2)
                #     performance_sim.append(1/(1+math.sqrt(sum)))
                # #     cosine = np.dot(true_conf_performance,predicted_conf_performance)/(norm(true_conf_performance)*norm(predicted_conf_performance))
                # #     cosine_sim.append(cosine)
                # #     pearson_corr.append(np.corrcoef(true_conf_performance, predicted_conf_performance)[0, 1])
                # print(round(np.mean(performance_sim),3), budget, dim, mode, modAlgo)
                # round(np.mean(cosine_sim),3), round(np.mean(pearson_corr),3), 


    #             break
    #         break
    #     break
    # break

<bound method NDFrame.head of true_conf
5       55
9      131
25     119
46      44
63      13
      ... 
276    279
293     19
308    310
322    320
323    321
Name: predicted_conf, Length: 324, dtype: int64>
<bound method NDFrame.head of true_conf
5       95
9      117
25      33
46      44
63     143
      ... 
276    316
293    291
308    306
322    320
323    301
Name: predicted_conf, Length: 324, dtype: int64>
<bound method NDFrame.head of true_conf
5       55
9       10
25      27
46      24
63     124
      ... 
276    314
293    247
308    306
322    320
323    315
Name: predicted_conf, Length: 324, dtype: int64>
<bound method NDFrame.head of true_conf
5        0
9        6
25      82
46      32
63      24
      ... 
276    317
293    305
308    269
322    214
323    251
Name: predicted_conf, Length: 324, dtype: int64>
<bound method NDFrame.head of true_conf
5       67
9       87
25      45
46       7
63      73
      ... 
276    312
293    239
308    322
322    249
323    305